# Check whether a match's charity headquarters (if available) match a shell company's address

In [14]:
#Imports
import pandas as pd

#spark
import findspark
findspark.init('/opt/spark/spark-2.3.2-bin-hadoop2.7/')

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import udf
from pyspark.sql.functions import split
from pyspark.sql.functions import explode

from pyspark.sql.types import StringType
from pyspark.sql.types import TimestampType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()

import networkx as nx

from operator import itemgetter
import matplotlib.pyplot as plt
import collections
from community import community_louvain
from networkx.algorithms.community.centrality import girvan_newman
import itertools
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [15]:
def get_match_file(leak, charity):
    return '../generated/matches/' + leak + '_' + charity + '_matches.csv'

#Load matches
panama_wiki_matches = spark.read.csv(get_match_file('panama', 'wikipedia'), header=True)
panama_ingo_matches = spark.read.csv(get_match_file('panama', 'INGO'), header=True)
panama_forbes_matches = spark.read.csv(get_match_file('panama', 'forbes'), header=True)

matches = panama_wiki_matches.union(panama_forbes_matches).union(panama_ingo_matches).drop('_c0')

#Load edges
edges = spark.read.csv('../data/panama/panama_papers.edges.csv', header=True)

address_nodes = spark.read.csv('../data/panama/panama_papers.nodes.address.csv', header=True)

In [16]:
print('All edges: ', edges.count())
address_edges = edges.rdd.filter(lambda r: r[1] == 'registered_address')
print('Address edges: ', address_edges.count())

All edges:  674102
Address edges:  151105


In [20]:
def filter_edges(edges, nodes):
    '''Given a set of nodes, returns the edges connected to those nodes'''
    entity_ids = nodes.map(lambda r: r[0]).collect()
    return edges.filter(lambda r: r[0] in entity_ids or r[2] in entity_ids)

entity_address_edges = filter_edges(address_edges, matches.rdd)


In [22]:
address_edges.take(5)

[Row(START_ID='10000035', TYPE='registered_address', END_ID='14095990', link='registered address', start_date=None, end_date=None, sourceID='Panama Papers', valid_until=None),
 Row(START_ID='10000044', TYPE='registered_address', END_ID='14091035', link='registered address', start_date=None, end_date=None, sourceID='Panama Papers', valid_until=None),
 Row(START_ID='10000055', TYPE='registered_address', END_ID='14095990', link='registered address', start_date=None, end_date=None, sourceID='Panama Papers', valid_until=None),
 Row(START_ID='10000064', TYPE='registered_address', END_ID='14091429', link='registered address', start_date=None, end_date=None, sourceID='Panama Papers', valid_until=None),
 Row(START_ID='10000089', TYPE='registered_address', END_ID='14098253', link='registered address', start_date=None, end_date=None, sourceID='Panama Papers', valid_until=None)]